In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

In [2]:
cam = 0
model_version = "my_model_13.0.keras"

Im_Width = 128
Im_Height = 128

In [3]:
def crop_image(frame, x, y, w, h):
    """
    Crops a region of interest (ROI) from an image.

    Args:
        frame (np.array): Input image.
        x (int): X-coordinate of the top-left corner of the ROI.
        y (int): Y-coordinate of the top-left corner of the ROI.
        w (int): Width of the ROI.
        h (int): Height of the ROI.

    Returns:
        np.array: Cropped image.
    """
    return frame[y:y+h, x:x+w]

In [4]:
def measure_brightness(frame):
    """
    Measures the brightness of an image by calculating the average pixel intensity.

    Args:
        frame (np.array): Input image in BGR format.

    Returns:
        float: Average brightness of the image.
    """
    cropped_frame = crop_image(frame, 120, 0, 400, 250)
    gray = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    brightness = np.mean(gray)  # Compute the mean intensity
    return brightness

In [5]:
def select_best_frame(frame_buffer):
    """
    Selects the best frame based on Brightness.

    Args:
        frame_buffer (list): List of frames.

    Returns:
        np.array: The best frame.
    """
    best_frame = None
    best_score = -1  # Initialize with a very low score

    for frame in frame_buffer:
        # Measure edge richness
        score = measure_brightness(frame)
        if score > best_score:
            best_score = score
            best_frame = frame

    return best_frame


In [6]:
def process_frame(frame, target_size):
    """
    Processes a webcam frame for model prediction.
    
    Args:
        frame (np.array): Captured frame from the webcam.
        target_size (tuple): Target size for resizing the frame (width, height).

    Returns:
        np.array: Preprocessed image ready for model prediction.
    """
    # Resize the frame to the target size
    resized_frame = cv2.resize(frame, target_size)

    # Normalize pixel values to [0, 1]
    normalized_frame = resized_frame / 255.0

    # Convert to NumPy array and add batch dimension
    frame_array = img_to_array(normalized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)  # Shape: (1, height, width, channels)

    return frame_array

In [7]:
def predict_frame(frame, model, target_size):
    """
    Predicts the class of a single frame using the model.

    Args:
        frame (np.array): Captured frame from the webcam.
        model: Loaded Keras model.
        target_size (tuple): Target size for resizing the frame.

    Returns:
        str: Predicted class label.
        float: Confidence score.
    """
    # Preprocess the frame
    frame_array = process_frame(frame, target_size)

    # Make a prediction
    predictions = model.predict(frame_array)  # Outputs probabilities for each class
    predicted_class = np.argmax(predictions[0])  # Get the index of the class with the highest probability
    confidence = predictions[0][predicted_class]  # Confidence score for the predicted class

    # Map class indices to labels
    class_labels = {0: "Defective", 2: "Non-Defective", 1: "No Bottle"}  # Update this to match your classes
    predicted_label = class_labels[predicted_class]

    print(f"Raw Predictions: {predictions[0]}")
    print(f"Predicted Class: {predicted_label}, Confidence: {confidence:.2f}")

    
    return predicted_label, confidence

In [8]:
def capture(cam, model_path, target_size=(Im_Width, Im_Height)):
    cap = cv2.VideoCapture(cam)  # Open webcam
    frame_buffer = []
    max_frames = 30  # Number of frames to evaluate

    # Load the model
    print("Loading model...")
    model = load_model(model_path)
    print("Model loaded successfully.")

    while True:
         # Set desired frame rate
        desired_fps = 30  # Set this to your camera's maximum frame rate
        cap.set(cv2.CAP_PROP_FPS, desired_fps)

        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame. Exiting...")
            break

        frame_buffer.append(frame)

        # Process the buffer when it's full
        if len(frame_buffer) == max_frames:
            best_frame = select_best_frame(frame_buffer)
            frame_buffer = []  # Clear the buffer

            # Process and predict
            feedback, confidence = predict_frame(best_frame, model, target_size)
            print(f"Prediction: {feedback} (Confidence: {confidence:.2f})")

            cv2.putText(best_frame, f"Prediction: {feedback}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(best_frame, f"Confidence: {confidence:.2f}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # Display the best frame
            cv2.imshow("Best Frame", best_frame)
            

            # Add further processing (e.g., prediction)
            # predicted_label, confidence = predict_frame(best_frame, model, target_size=(128, 128))
            # print(f"Prediction: {predicted_label}, Confidence: {confidence:.2f}")

        # Exit on 'ESC' key
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

In [9]:
if __name__ == "__main__":
    model_path = model_version  # Path to your saved model
    capture(cam, model_path, target_size=(Im_Width, Im_Height))

Loading model...
Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 825ms/step
Raw Predictions: [0.9882641  0.00643971 0.00529621]
Predicted Class: Defective, Confidence: 0.99
Prediction: Defective (Confidence: 0.99)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Raw Predictions: [0.98917925 0.00102846 0.00979222]
Predicted Class: Defective, Confidence: 0.99
Prediction: Defective (Confidence: 0.99)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Raw Predictions: [9.9511433e-01 8.7270379e-04 4.0129176e-03]
Predicted Class: Defective, Confidence: 1.00
Prediction: Defective (Confidence: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Raw Predictions: [9.9653059e-01 2.7581188e-03 7.1131228e-04]
Predicted Class: Defective, Confidence: 1.00
Prediction: Defective (Confidence: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Raw Predictions: [9.9547726e-01 3.6992850e-03 8.2340610e-04]
Predicted Class: Defective, Confidence: 1.00
Prediction: Defective (Confidence: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Raw Predic